In [32]:
import pandas as pd
import re
import numpy as np

## FUZZY
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
## nomalice parameters
from unidecode import unidecode

fileOut = "fuzzy_partial_ratio_about_boots.xlsx"

In [33]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [34]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 

dfa = df[df["about"].notna()] #DATAFRAME DE EDUCATION
dfa = dfa.reset_index(drop = True) #REINICIAR INDICE

In [35]:
dfa.columns

Index(['id', 'scraping_services_id', 'linkedin_id', 'linkedin_profile_url',
       'last_name', 'first_name', 'v_extension', 'location', 'job_title',
       'company', 'job_history', 'snapshots', 'skills', 'education', 'about',
       'updated', 'middle_name', 'num_connections', 'interests', 'volunteered',
       'certifications', 'languages', 'recommendations', 'publications',
       'patents', 'industry', 'titles', 'company_name_complete',
       'job_title_complete'],
      dtype='object')

In [36]:
#EL NUMERO DE NO NAS PARA DE 2500 A 1423
dfa.shape

(1423, 29)

In [37]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'dev f\\.'
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech', 
 'bictia|bi.?tia', 
 'talento[ -]?digital']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE ABOUT

In [38]:
#CONVERTIR EN STRING
dfa["about"] =  dfa["about"].str.lower()

In [39]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'iCreate Coding Academy',
 'IJALTI',
 'Iron Hack',
 'Jóvenes a Programar',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'Make it Real Camp',
 'MindHub ',
 'Muktek Academy',
 'Nivel Pro',
 'Noukod',
 'Plataforma 5',
 'Programa Valentina',
 'Pygmalion',
 'Reprograma',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)',
 'BICTIA',
 'Talento Digital']

In [40]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'pygmalion'
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia', 
 'talento[ -]?digital$'] #

In [41]:
#DADO QUE AL ABOUT NO TIENE NINGUNA ESTRUCTURA, SOLO RESTA BUSCAR EN LOS STRINGS DE CADA INDIVIDUO
c = []
for boot in range(len(lista_boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfa.about)): #PARA CADA INDIVIDUO
        bootCAMP = lista_boots[boot]
        bootCAMP = bootCAMP.lower()
        bootCAMP = unidecode(bootCAMP)
        norEdu =  unidecode(dfa.about[i])
        ## se hace un aproach a fuzzy
        wordLen = len(dfa.about[i])
        ratio = fuzz.partial_ratio(bootCAMP,norEdu)
        #ratio = fuzz.token_sort_ratio(bootCAMP,norEdu)
        #ratio = fuzz.token_set_ratio(bootCAMP,norEdu)
        #print(ratio)
        if ratio>85 and wordLen >=5:
            c.append((dfa.id[i], lista_boots[boot], lista_boots[boot],ratio))
           
            
            print(f"{dfa['job_title'][i]}---{ratio}---{lista_boots[boot]}--- \n {dfa.about[i]} \n ------------------------------------------\n")
        """ A = re.findall(boots[boot], dfa.about[i])
        if len(A) > 0:
            c.append((dfa.id[i], boots[boot], lista_boots[boot]))
            print(f"{dfa['job_title'][i]}---{A[0]}---{boots[boot]}--- \n {dfa.about[i]} \n ------------------------------------------\n") """


e tanto en una empresa como en una clínica, mi ilusión, mi motivación y vocación se mantienen con mucha fuerza.

 el ser humano esta en constante cambio y evolución, y no hay una manera correcta de poder vivir, por lo que podemos equivocarnos. pero creo, que la psicología tiene esa ''magia'' que da la oportunidad de realzar el potencial individual y grupal, enriquecer a las personas y ofrecerles una nueva oportunidad de cambio en su vida.
 podemos encontrar psicología en muchos rinconcitos de nuestra vida, y no hace falta que sea reconocida solo como una herramienta para personas con diagnósticos clínicos, si no que puede ofrecer una orientación, una oportunidad de poder enfocar las preocupaciones , malestares, así como enfocarnos a nuevos objetivos que proporcionen a las personas una mejora, y unos resultados favorables en su vida. 
 ------------------------------------------

Bartender---100---Nivel Pro--- 
 me considero una persona ambiciosa a nivel profesional, luchadora y con gana

Parece que el desempeño en este caso es positivo. En los perfiles de about parece haber información relevante.

In [42]:
#CREAR DATAFRAME
c = list(set(c))
educ_boots = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp',"ratio"])

In [43]:
len(c)
#14 POTENCIALES PERFILES DESDE ABOUT

73

In [44]:
#HACER MERGE CON EL RESTO DE COLUMNAS
about_boots = pd.merge(educ_boots, dfa, "left", "id")
about_boots.to_excel(fileOut, index = False)

In [ ]:
about_boots